In [10]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re, random, itertools, time, statistics, operator
from collections import deque
from tqdm import tqdm

In [2]:
# Reading the data

path = './data/wikigraph_reduced.csv' #insert your own path of the file 'wikigraph_reduced.csv'
dataset=pd.read_csv(path, sep='\t')
dataset.rename(columns={'Unnamed: 0':'old_index'}, inplace=True)

In [3]:
# Building the graph 

G = nx.convert_matrix.from_pandas_edgelist(dataset,"0","1",create_using=nx.DiGraph())
# G = nx.DiGraph()
# for i in range(0, len(dataset)):
#     G.add_node(dataset.iloc[i][1])
#     G.add_node(dataset.iloc[i][2])
#     G.add_edge(dataset.iloc[i][1],dataset.iloc[i][2])

In [4]:
def build_category_and_articles_dictionary():
    '''
    Builds a dictionary where the keys are categories and the values are the articles associate to the category.
    Every article is associated to a single random category.
    
    :return: dictionary
    '''
    
    # Storing the articles and categories in a dataframe
    categories_dataset = pd.read_csv("./data/wiki-topcats-categories.txt",sep=';',names=['Category','Articles'],header=None)
    categories_dataset['Category'] = categories_dataset.Category.apply(lambda x: x.split(':')[1])
    categories_dataset['Articles'] = categories_dataset.Articles.apply(lambda x : list(map(int,x.split())))
    
    # Find the number of articles for each category
    categories_dataset['Number_of_articles'] = [len(x) for x in categories_dataset['Articles']]

    # Removes the categories whose number of articles is less than 5000 and more than 30000
    categories_dataset = categories_dataset.loc[(categories_dataset['Number_of_articles']>5000) & (categories_dataset['Number_of_articles']<30000)]
    
    
    df_article = pd.concat([pd.DataFrame(data = {'Articles': categories_dataset.loc[i].Articles, 
                                             'Category': categories_dataset.loc[i].Category}) for i in categories_dataset.index], 
                       ignore_index=True)
    
    # Choose a single random category for an article
    rand_article = df_article.groupby('Articles').sample(random_state=1)
    df_article_category = pd.DataFrame(data={'Articles': rand_article.Articles, 'Category': rand_article.Category})
    
    df_article_category = df_article_category.reset_index()
    categories = df_article_category.Category.unique().tolist()
    
    articles = []
    for category in categories:
        articles.append(df_article_category[df_article_category.Category == category].Articles.values.tolist())
        
    df_article = dict(zip(categories, articles))
            
    return df_article

In [15]:
def get_shortest_paths_from_two_categories(C0):
    '''
    Given an arbitrary category C0 as input, returns the list of remaning categories sorted by their distance from C0. 
    In particular, the distance between two categories is defined as distance(C0, Ci) = median(ShortestPath(C0, Ci)),
    where ShortestPath(C0, Ci) is the set of shortest paths from each pair of nodes in the two categories.
    
    For simplicity we discard categories whose nodes aren't connected at all with the nodes of the inputed category.
    This implies that the function considers as reachable a category who's at least one node is connected to the inputed category.

    :param C0: Chosen category
    :return: Dictionary where the keys are the remaining categories and the values the medians.
    '''
    
    if C0 not in d:
        print('The category ' + C0 + ' doesn\'t exist.')
        return

    shortest_path_lengths = {}

    # Compute path lengths from all source nodes to all other nodes in the graph
    
    #for i, source_node in tqdm(enumerate(d[C0])):
    for source_node in tqdm(d[C0]):
        #if i == 20:
        #    break
        for target_node,path_length in bfs_lenght_shortest_path(G,source_node).items():
            if target_node not in shortest_path_lengths:
                shortest_path_lengths[target_node] = []
            shortest_path_lengths[target_node].append(path_length)
    
    result = {}
    
    for category_name, target_nodes in d.items():
        #print(f"Iteration {category_name}")
        
        # Jump to next iteration
        if category_name == C0:
            continue

        # List of shortest path of a category
        category_paths_lengths = [
            path_length
            for target_node in target_nodes
            for path_length in shortest_path_lengths.get(target_node,[])
        ]
        #print(len(category_paths_lengths))
        
        # Calculating median for the set of shortest paths of one category
        try:
            result[category_name] = statistics.median(category_paths_lengths)
            
        # Excluding categories whose nodes aren't connected at all with the nodes of the inputed category
        except:
            pass
        
    result = sorted(result.items(), key=operator.itemgetter(1))
   
    return result

In [11]:
def bfs_lenght_shortest_path(G, source_node):
    '''
    Computes the shortest paths from a source node to all the other nodes of the graph.
    We are considering only the nodes that exist in the graph for simplicity.
    
    :param G: Graph
    :param source_node: An article/node associated to the chosen category, C0
    :return: Dictionary containing target nodes as keys and length of the shortest paths as values
    '''
    visited = {source_node: 0}
    queue = deque([source_node])
    while len(queue) > 0:
        node = queue.popleft()
        distance = visited[node]
        
        try:
            for adjacent in G.neighbors(node):
                if adjacent not in visited:
                    queue.append(adjacent)
                    visited[adjacent] = distance+1
                    
        # We are considering only the nodes that exist in the graph for simplicity
        except: 
            pass
    return visited

In [16]:
print("Starting time: " + time.asctime(time.localtime(time.time()))) 

list_of_categories = get_shortest_paths_from_two_categories("Asteroids_named_for_people")

end = "Ending time: " + time.asctime(time.localtime(time.time()))

list_of_categories

  0%|                                                                                         | 0/3075 [00:00<?, ?it/s]

Starting time: Mon Jan 11 19:06:51 2021


100%|██████████████████████████████████████████████████████████████████████████████| 3075/3075 [12:50<00:00,  3.99it/s]


Iteration English-language_films
25691040
Iteration Harvard_University_alumni
3016449
Iteration Major_League_Baseball_pitchers
4728720
Iteration Debut_albums
1511640
Iteration Year_of_death_missing
663483
Iteration Black-and-white_films
10100400
Iteration Year_of_birth_missing
761545
Iteration Place_of_birth_missing_(living_people)
2658481
Iteration American_films
15510840
Iteration American_film_actors
19268280
Iteration American_television_actors
13032480
Iteration Indian_films
3130440
Iteration Asteroids_named_for_people
Iteration Main_Belt_asteroids
164365
Iteration Association_football_midfielders
2528520
Iteration Association_football_defenders
1960800
Iteration The_Football_League_players
5043360
Iteration Association_football_forwards
2615160
Iteration English_footballers
4879200
Iteration Members_of_the_United_Kingdom_Parliament_for_English_constituencies
13575120
Iteration Rivers_of_Romania
15063960


[('Main_Belt_asteroids', 5),
 ('English-language_films', 9.0),
 ('Harvard_University_alumni', 9),
 ('Black-and-white_films', 9.0),
 ('American_films', 9.0),
 ('American_film_actors', 9.0),
 ('American_television_actors', 9.0),
 ('Debut_albums', 10.0),
 ('Year_of_death_missing', 10),
 ('Year_of_birth_missing', 10),
 ('Place_of_birth_missing_(living_people)', 10),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 10.0),
 ('Major_League_Baseball_pitchers', 11.0),
 ('Indian_films', 12.0),
 ('Association_football_midfielders', 12.0),
 ('The_Football_League_players', 12.0),
 ('Association_football_forwards', 12.0),
 ('English_footballers', 12.0),
 ('Association_football_defenders', 13.0),
 ('Rivers_of_Romania', 13.0)]

In [17]:
end

'Ending time: Mon Jan 11 19:20:07 2021'